Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = './data/notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [18]:
train_subset = 10000
regulization = 0.1

graph = tf.Graph()
with graph.as_default():
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)
    
    #variables
    W = tf.Variable(tf.truncated_normal([image_size*image_size, num_labels]))
    b = tf.Variable(tf.zeros([num_labels]))
    
    # Training computation
    logits = tf.matmul(tf_train_dataset, W) + b
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + tf.nn.l2_loss(W) * regulization
    
    # Optimizer
    optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
    
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(
        tf.matmul(tf_valid_dataset, W) + b)
    test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, W) + b)
    

In [19]:
num_steps = 801

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  for step in range(num_steps):
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Initialized
Loss at step 0: 323.861237
Training accuracy: 10.4%
Validation accuracy: 14.6%
Loss at step 100: 1.010379
Training accuracy: 80.9%
Validation accuracy: 80.7%
Loss at step 200: 0.997735
Training accuracy: 80.9%
Validation accuracy: 80.8%
Loss at step 300: 0.997073
Training accuracy: 81.0%
Validation accuracy: 80.7%
Loss at step 400: 0.996906
Training accuracy: 81.0%
Validation accuracy: 80.8%
Loss at step 500: 0.996855
Training accuracy: 81.0%
Validation accuracy: 80.8%
Loss at step 600: 0.996837
Training accuracy: 81.0%
Validation accuracy: 80.8%
Loss at step 700: 0.996831
Training accuracy: 81.0%
Validation accuracy: 80.8%
Loss at step 800: 0.996829
Training accuracy: 81.0%
Validation accuracy: 80.8%
Test accuracy: 87.7%


Trying with the hidden layer and regulization

In [31]:
batch_size = 128
hidden_size = 1024
num_steps = 3001
regulization = 0.001

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  biases1 = tf.Variable(tf.zeros([hidden_size]))
  
  weights2 = tf.Variable(
  tf.truncated_normal([hidden_size, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

    
  # Training computation.
  logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + tf.nn.l2_loss(weights1) * regulization + tf.nn.l2_loss(weights2) * regulization
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2
  test_prediction = tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2

In [32]:
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 677.378174
Minibatch accuracy: 3.9%
Validation accuracy: 39.1%
Minibatch loss at step 500: 195.423386
Minibatch accuracy: 82.0%
Validation accuracy: 79.5%
Minibatch loss at step 1000: 117.777588
Minibatch accuracy: 79.7%
Validation accuracy: 80.4%
Minibatch loss at step 1500: 68.837517
Minibatch accuracy: 80.5%
Validation accuracy: 83.9%
Minibatch loss at step 2000: 41.615551
Minibatch accuracy: 85.9%
Validation accuracy: 85.1%
Minibatch loss at step 2500: 25.316645
Minibatch accuracy: 84.4%
Validation accuracy: 86.3%
Minibatch loss at step 3000: 15.506633
Minibatch accuracy: 89.1%
Validation accuracy: 87.2%
Test accuracy: 92.7%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [34]:
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % 10
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 667.975891
Minibatch accuracy: 12.5%
Validation accuracy: 36.4%
Minibatch loss at step 500: 191.078964
Minibatch accuracy: 100.0%
Validation accuracy: 66.8%
Minibatch loss at step 1000: 115.880966
Minibatch accuracy: 100.0%
Validation accuracy: 66.8%
Minibatch loss at step 1500: 70.276596
Minibatch accuracy: 100.0%
Validation accuracy: 66.8%
Minibatch loss at step 2000: 42.619480
Minibatch accuracy: 100.0%
Validation accuracy: 66.8%
Minibatch loss at step 2500: 25.846918
Minibatch accuracy: 100.0%
Validation accuracy: 66.7%
Minibatch loss at step 3000: 15.675076
Minibatch accuracy: 100.0%
Validation accuracy: 66.7%
Test accuracy: 73.3%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [52]:
batch_size = 128
hidden_size = 1024
num_steps = 3001
regulization = 0.001

keep = 0.5

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)

  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_size]))
  biases1 = tf.Variable(tf.zeros([hidden_size]))
  
  weights2 = tf.Variable(
  tf.truncated_normal([hidden_size, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

    
  # Training computation.
  logits = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, tf.nn.dropout(weights1, keep)) + biases1), weights2) + biases2

  predict = tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, weights1) + biases1), weights2) + biases2

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + tf.nn.l2_loss(weights1) * regulization + tf.nn.l2_loss(weights2) * regulization
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(predict)
  valid_prediction = tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2
  test_prediction = tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2

In [53]:
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 495.538788
Minibatch accuracy: 12.5%
Validation accuracy: 30.2%
Minibatch loss at step 500: 110.382683
Minibatch accuracy: 75.8%
Validation accuracy: 79.1%
Minibatch loss at step 1000: 103.744316
Minibatch accuracy: 81.2%
Validation accuracy: 79.9%
Minibatch loss at step 1500: 71.304550
Minibatch accuracy: 76.6%
Validation accuracy: 79.0%
Minibatch loss at step 2000: 37.568306
Minibatch accuracy: 76.6%
Validation accuracy: 77.1%
Minibatch loss at step 2500: 54.534752
Minibatch accuracy: 76.6%
Validation accuracy: 79.4%
Minibatch loss at step 3000: 31.570805
Minibatch accuracy: 72.7%
Validation accuracy: 79.7%
Test accuracy: 86.5%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [14]:
batch_size = 128
num_steps = 3001
regulization = 0.000

drop_rate = 0.5

n_input = 784 # notMNIST data input (img shape: 28*28)
n_classes = 10 # notMNIST total classes (A-J)

h1 = 1000 # 1st layer number of features
h2 = 500 # 2nd layer number of features
h3 = 200 # 3rd layer number of features
h4 = 64 # 3rd layer number of features
h5 = 32 # 3rd layer number of features

r1 = 0.0002
r2 = 0.0002
r3 = 0.0002
r4 = 0.0002
r5 = 0.0002
ro = 0.0002

graph = tf.Graph()
with graph.as_default():

    # Input data. For the training data, we use a placeholder that will be fed
    # at run time with a training minibatch.
    tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
    tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_test_dataset = tf.constant(test_dataset)

    # Variables.
    W1 = tf.Variable(tf.truncated_normal([n_input, h1]))
    b1 = tf.Variable(tf.zeros([h1]))
  
    W2 = tf.Variable(tf.truncated_normal([h1, h2]))
    b2 = tf.Variable(tf.zeros([h2]))
  
#     W3 = tf.Variable(tf.truncated_normal([h2, h3]))
#     b3 = tf.Variable(tf.zeros([h3]))
  
#     W4 = tf.Variable(tf.truncated_normal([h3, h4]))
#     b4 = tf.Variable(tf.zeros([h4]))
  
#     W5 = tf.Variable(tf.truncated_normal([h4, h5]))
#     b5 = tf.Variable(tf.zeros([h5]))
  
    
    Wo = tf.Variable(tf.truncated_normal([h2, n_classes]))
    bo = tf.Variable(tf.zeros([n_classes]))
    
    # Training computation.
    hl1 = tf.nn.relu(tf.matmul(tf_train_dataset, tf.nn.dropout(W1, drop_rate)) + b1)
    hl2 = tf.nn.relu(tf.matmul(hl1, tf.nn.dropout(W2, drop_rate)) + b2)
#     hl3 = tf.nn.relu(tf.matmul(hl2, tf.nn.dropout(W3, drop_rate)) + b3)
#     hl4 = tf.nn.relu(tf.matmul(hl3, tf.nn.dropout(W4, drop_rate)) + b4)
#     hl5 = tf.nn.relu(tf.matmul(hl4, tf.nn.dropout(W5, drop_rate)) + b5)

    logits = tf.matmul(hl2, Wo) + bo

#     predict = tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_train_dataset, W1) + b1), W2) + b2), W3) + b3), Wo) + bo
# + tf.nn.l2_loss(W2) * r2 + tf.nn.l2_loss(W3) * r3 + tf.nn.l2_loss(W4) * r4 + tf.nn.l2_loss(W5) * r5
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits)) + tf.nn.l2_loss(W1) * r1 + tf.nn.l2_loss(Wo) * ro + + tf.nn.l2_loss(W2) * r2
    
    # Optimizer.
    optimizer = tf.train.AdamOptimizer().minimize(loss)
#     optimizer = tf.train.GradientDescentOptimizer(0.01).minimize(loss)
    
    # Predictions for the training, validation, and test data.
#     train_prediction = tf.nn.softmax(predict)
    train_prediction = tf.nn.softmax(logits)
    valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1), W2) + b2), Wo) + bo)
    test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, W1) + b1), W2) + b2), Wo) + bo)
  
#     valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, W1) + b1), W2) + b2), W3) + b3), W4) + b4), W5) + b5), Wo) + bo)
#     test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, W1) + b1), W2) + b2), W3) + b3), W4) + b4), W5) + b5), Wo) + bo)

#     valid_prediction = tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2
#     test_prediction = tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2


TypeError: bad operand type for unary +: 'Tensor'

In [13]:
with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print(np.average(valid_labels, axis=0))
  for step in range(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    
    if (step % 200 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

[ 0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1  0.1]
Minibatch loss at step 0: 484.882355
Minibatch accuracy: 8.6%
Validation accuracy: 12.0%
Minibatch loss at step 200: 146.516418
Minibatch accuracy: 73.4%
Validation accuracy: 76.1%
Minibatch loss at step 400: 139.745682
Minibatch accuracy: 74.2%
Validation accuracy: 79.6%
Minibatch loss at step 600: 172.592072
Minibatch accuracy: 64.1%
Validation accuracy: 79.8%
Minibatch loss at step 800: 158.158279
Minibatch accuracy: 65.6%
Validation accuracy: 81.2%
Minibatch loss at step 1000: 141.355011
Minibatch accuracy: 75.8%
Validation accuracy: 80.9%
Minibatch loss at step 1200: 144.479813
Minibatch accuracy: 75.8%
Validation accuracy: 81.5%
Minibatch loss at step 1400: 143.676605
Minibatch accuracy: 69.5%
Validation accuracy: 82.2%
Minibatch loss at step 1600: 127.482689
Minibatch accuracy: 73.4%
Validation accuracy: 82.2%
Minibatch loss at step 1800: 125.789627
Minibatch accuracy: 71.9%
Validation accuracy: 82.8%
Minibatch loss at step 200